# **Lab 3: Iterative methods**
**Sanskar Gupta**

# **Abstract**
This notebook is the third lab in the course DD2363 Methods in Scientific Computing. This lab involves iterative methods for solving the equation Ax=b and f(x)=0.


#**About the code**

In [15]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2021 Sanskar Gupta (sanskar@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

#**Environment Setup**

In [16]:
#Loading necessary stuffs
from google.colab import files

import time
import numpy as np
import sympy as sp
from sympy import *
import random

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

import unittest


#**Introduction**

This equation can be solved by a variety of different methods, imcluding direct methods and iterative methods. In this notebook following iterative methods will be implemented:  

*   Jacobi iteration
*   Gauss-Seidel iteration
*   Newton's method for scalar nonlinear equation
*   GMRES

Additionally, Newton's method for vector nonlinear equations will be implemented.


#**Methods**

##**Jacobi Iteration**
Jacobi iteration works through matrix splitting, where $A = A_1 + A_2$. $A_1$ is the diagonal elements of $A$ while $A_2 = A - A_1$.

$x^{(k+1)} = (I-D^{-1}A)x^{(k)} + D^{-1}b$

In [17]:
def getJacobi(A, b, tolerance):

  n = A.shape[0]
  I = np.identity(n)
  D = np.diag(np.diag(A))
  D_inverse = np.linalg.inv(D)
  
  constant = D_inverse.dot(b)
  x = constant  #initializing x with the constant(c)

  while np.linalg.norm(A.dot(x) - b) > tolerance:
    x = (I-np.matmul(D_inverse,A)).dot(x) + constant #just following the equation above

  return x

In [18]:
#Test
class TestJacobi(unittest.TestCase):

  def test(self):
    for i in range(0,10):
      row = random.randrange(30) + 10
      A = np.random.rand(row,row)
      x = np.random.rand(row)
      b = A.dot(x)

      #  Richardson Preconditioning for convergence, here B is the approximate inverse of A
      B = np.linalg.inv(A)
      B = B * 0.9
      A = np.matmul(B,A)
      b = B.dot(b)

      x2 = getJacobi(A,b,1e-9)
    
      # ||Ax-b|| = 0
      np.testing.assert_almost_equal(np.linalg.norm(A.dot(x2)-b), 0, decimal=7)

      # ||x-x2|| = 0
      np.testing.assert_almost_equal(np.linalg.norm(x-x2), 0, decimal=7)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.021s

OK


##**Gauss-Seidel iteration**

Gauss-Seidel iterationinvolves  matrix splitting, but instead of a diagonal matrix, $A_1 = L$ where L is a lower triangluar matrix.

$x^{(k+1)} = (I-L^{-1}A)x^{(k)} + L^{-1}b$

In [19]:
def getGaussSeidel(A, b, tolerance):

  n = A.shape[0]
  I = np.identity(n)
  L = np.tril(A)
  L_inverse = np.linalg.inv(L)
  constant = L_inverse.dot(b)
  x = constant

  while np.linalg.norm(A.dot(x) - b) > tolerance:
    x = (I-np.matmul(L_inverse,A)).dot(x) + constant #just following the equation above

  return x

In [20]:
#Test
class TestGaussSeidel(unittest.TestCase):

  def test(self):
    for i in range(0,10):
      row = random.randrange(30) + 10
      A = np.random.rand(row,row)
      x = np.random.rand(row)
      b = A.dot(x)

      #  Richardson Preconditioning for convergence, here B is the approximate inverse of A
      B = np.linalg.inv(A)
      B = B * 0.9
      A = np.matmul(B,A)
      b = B.dot(b)

      x2 = getGaussSeidel(A,b,1e-9)
    
      # ||Ax-b|| = 0
      np.testing.assert_almost_equal(np.linalg.norm(A.dot(x2)-b), 0, decimal=7)
      
      # ||x-x2|| = 0
      np.testing.assert_almost_equal(np.linalg.norm(x-x2), 0, decimal=7)
     

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.027s

OK


##**Newton's method for scalar nonlinear equation**




Newton's method is based on the first order derivative of a nonlinear function.

$x^{(k+1)} = x^{(k)} - f^{'}(x^{(k)})^{-1} f(x^{(k)})$

In [21]:
def scalarNewtonMethod(f, y, tolerance):

  # Using sympy to calculate f'
  x = Symbol('x')
  df = f.diff(x)
  f = lambdify(x, f)
  df = lambdify(x, df)

  while np.abs(f(y)) > tolerance:
    y -= f(y)/df(y)

  return y

In [22]:
#Test
from numpy import cos, sin, pi, exp
class Test(unittest.TestCase):

  def testRandom(self):
    for i in range(0,1000):

      # Generating a random function using sympy
      exponents = [random.randint(1,10) for i in range(3)]
      x = Symbol('x')
      f = x**exponents[0] + x**exponents[1] - random.randint(0,10000)
      y = scalarNewtonMethod(f, 1, 1e-7)
      f = lambdify(x, f)

      # |f(x)|
      np.testing.assert_almost_equal(np.abs(f(y)), 0, decimal=5)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 4.193s

OK


## GMRES method

Generalised minimal residual, where we solve for $\displaystyle \min_{y \in \mathbb{R}^k} ||b-AQ_ky||$ where $Q_k$ is an orthonormal basis for $K_k$. Through Arnoldi iteration we get $\displaystyle \min_{y \in \mathbb{R}^k} || \space ||b||e_1-H_ky||$.

We get the approximation through $x^{(k)} = Q_ky$

In [23]:
def getArnoldiIteration(A, b, k):
  n = A.shape[0]
  Q = np.zeros(shape=(n,k+1))
  H = np.zeros(shape=(k+1,k))
  v = np.zeros(n)

  Q[:,0] = b / np.linalg.norm(b)
  for j in range(0,k):
    v = A.dot(Q[:,j])
    for i in range(0, j):
      H[i,j] = Q[:,i].dot(v)
      v = v - H[i,j] * Q[:,i]
    H[j+1,j] = np.linalg.norm(v)
    Q[:,j+1] = v / H[j+1,j]
  
  return Q, H

In [24]:
def getStandardBasis(k, i=0):
  return (np.array(range(k)) == i) * 1

In [25]:
def gmres(A, b, tolerance = 1e-10):
  k = 1
  r = None
  while r is None or np.linalg.norm(r) / np.linalg.norm(b) > tolerance:
    Q, H = getArnoldiIteration(A, b, k)
    y = np.linalg.lstsq(H, np.linalg.norm(b) * getStandardBasis(k + 1))[0]
    r = H.dot(y)
    r = np.linalg.norm(b) * getStandardBasis(k + 1) - r
    k += 1
  return Q[:, 0:k-1].dot(y)

In [26]:
class Test(unittest.TestCase):

  def test(self):
    for i in range(0,2):
      row = random.randrange(10) + 10
      A = np.random.rand(row,row)
      x = np.random.rand(row)
      b = A.dot(x)
      x2 = gmres(A,b,1e-6)
      # ||Ax-b|| = 0
      np.testing.assert_almost_equal(np.linalg.norm(A.dot(x2)-b), 0, decimal=2)

      # ||x-x2|| = 0
      np.testing.assert_almost_equal(np.linalg.norm(x-x2), 0, decimal=2)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  
...
----------------------------------------------------------------------
Ran 3 tests in 26.987s

OK


##**Newton's Method for vector nonlinear equation f(x)=0**

The idea of Newton’s method is that we have some approximation xi to the root and seek a new (and hopefully better) approximation xi+1 by approximating F(xi+1) by a linear function and solve the corresponding linear system of algebraic equations. We approximate the nonlinear problem F(xi+1)=0 by the linear problem

$ F(x^{(k)}) + J(x)(x^{(k+1)}-x^{(k)})$ = 0

In [27]:
import numpy as np

def newton_system_non_linear(F, J, x, eps):
    F_value = F(x)
    F_norm = np.linalg.norm(F_value, ord=2)  # l2 norm of vector
    iteration_counter = 0
    while abs(F_norm) > eps and iteration_counter < 100:
        delta = np.linalg.solve(J(x), -F_value)
        x = x + delta
        F_value = F(x)
        F_norm = np.linalg.norm(F_value, ord=2)
        iteration_counter += 1

    # Here, either a solution is found, or too many iterations
    if abs(F_norm) > eps:
        iteration_counter = -1
    return x, iteration_counter

In [28]:
def test_newton_system_non_linear():
    from numpy import cos, sin, pi, exp

    def F(x):
        return np.array(
            [x[0]**2 - x[1] + x[0]*cos(pi*x[0]),
             x[0]*x[1] + exp(-x[1]) - (1/x[0])])

    def J(x):
        return np.array(
            [[2*x[0] + cos(pi*x[0]) - pi*x[0]*sin(pi*x[0]), -1],
             [x[1] + 1/(x[0]**(2)), x[0] - exp(-x[1])]])

    expected = np.array([1, 0])
    tol = 1e-4
    x, n = Newton_system(F, J, x=np.array([2, -1]), eps=0.0001)
    print (n, x)
    error_norm = np.linalg.norm(expected - x, ord=2)
    assert error_norm < tol, 'norm of error =%g' % error_norm
    print ('norm of error =%g' % error_norm)

In [29]:
test_newton_system_non_linear()

4 [ 1.00000006e+00 -1.00943962e-06]
norm of error =1.01115e-06


#**Discussion**
Here all the methods works as expected , all the test cases seems to pass with varying tolerances and different assertion error values.
Also it was noted tat Gauss Seidel method was not converging without Richardson conditioning